# Module 6 Challenge

## Deliverable 1

In [5]:
# Import dependencies
from citipy import citipy
# from datetime import datetime
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
import time

# Import API keys from config.py file (one level up)
import sys
sys.path.insert(0, '../')
from config import weather_api_key, g_key

In [2]:
# Create list for holding cities
cities = []
list_length = 2000

# Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90,90,size=list_length)
lngs = np.random.uniform(-180,180,size=list_length)

lat_lngs = zip(lats,lngs)

coordinates = list(lat_lngs)

# Identify nearest city for each latitude-longitude combination
for coord in coordinates:
    city = citipy.nearest_city(coord[0], coord[1]).city_name

    # If city is unique, add it to cities list
    if city not in cities:
        cities.append(city)

print(f"Found {len(cities):,} cities.")

Found 736 cities.


In [3]:
# Import requests library
import requests

# Import API key
from config import weather_api_key

### The next cell takes approximately fifteen minutes to run. To avoid this lengthy delay on subsequent runs, disable the cells that follow up till the relevant note below by changing their Cell Types to Raw NBConvert (hotkey R).
# ▼Disable▼

In [ ]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
## record_count = 1
record_count = 0  # Allows us to move the incrementer to the start of the loop below
set_count = 1

# Loop through all cities in the list
for i, city in enumerate(cities):
    
    # Increment record count.
    record_count += 1

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and 50 <= i):
        set_count += 1
        record_count = 1
        print("-----------------------------")
        print(f"Preparing Set {set_count}.")
        print("-----------------------------")
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = 'https://api.openweathermap.org/data/2.5/weather?units=imperial&appid=' \
                + weather_api_key + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}", end="")
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()

        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_desc = city_weather["weather"][0]["description"]

        # Convert the date to ISO standard.
#         city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Append the city information into the city_data list.
        city_data.append({
            "City": city.title(),  # The `.title()` method converts the `city` string to title case
            "Country": city_country,
#             "Date": city_date,
            "Lat": city_lat,
            "Lng": city_lng,
            "Max Temp": city_max_temp,
            "Humidity": city_humidity,
            "Cloudiness": city_clouds,
            "Wind Speed": city_wind,
            "Current Description": city_desc
        })

        # Empty print statement only to provide a newline character following
        # the "Processing Record…" statement if the `try` succeeds.
        print()  

    # If an error is experienced, skip the city.
    except:
        print(" | City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
print(f'\nFound data for {len(city_data):,} cities.')

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame
df_city_data = pd.DataFrame(city_data)

df_city_data.head(10)

In [ ]:
# Reorder the columns
column_order = ['City','Country','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Current Description']

df_city_data = df_city_data[column_order]

df_city_data.head(10)

In [ ]:
# Create the output file (CSV)
# output_filepath = os.path.join('.','Weather_Database','WeatherPy_Database.csv')
output_filepath = os.path.join('.','WeatherPy_Database.csv')

# Export the City_Data into a CSV
df_city_data.to_csv(output_filepath, index_label='City_ID')

# ▲Disable▲

### The next cell recreates a DataFrame from a previous run of the cells between the Disable tags above. Enable it by changing its cell type to Code (hotkey Y).